# Simple Alfalfa Example

## Setup
1. [Alfalfa stack deployed](https://github.com/NREL/alfalfa/wiki/Deployment) locally with **at least two workers**
1. Create `small_office.zip` from `models`, `weather`, and `measures` folders and `small_office.osw`

## Notes
1. For Alfalfa v0.2.0, API reads and writes are implemented over the [Haystack API](https://project-haystack.org/doc/docHaystack/HttpApi), particularly the `read` and `pointWrite` endpoints.


In [16]:
import os
import datetime
import time
from alfalfa_client.alfalfa_client import AlfalfaClient

In [17]:
# Create a new client
ac = AlfalfaClient(url='http://localhost')

In [18]:
# Local paths to models for upload
model_paths = []
for i in range(180,182):
    model_paths.append(f'../CCTwin-scripts/bldg_models/apr22demo/zips/{i}.zip')

In [19]:
# Upload to Alfalfa 
model_ids = {}
for p in model_paths:
    print(p)
    # TODO add error handling
    model_ids[p] = ac.submit(p)
# m1_id = ac.submit(m1)
# m2_id = ac.submit(m2)


../CCTwin-scripts/bldg_models/apr22demo/zips/180.zip
Desired status: Stopped		Current status: 
Desired status: Stopped		Current status: Stopped
../CCTwin-scripts/bldg_models/apr22demo/zips/181.zip
Desired status: Stopped		Current status: 
Desired status: Stopped		Current status: 


In [20]:
print(model_ids)

{'../CCTwin-scripts/bldg_models/apr22demo/zips/180.zip': 'd5da77f6-c05a-11ec-aecf-acde48001122', '../CCTwin-scripts/bldg_models/apr22demo/zips/181.zip': 'd9b4e154-c05a-11ec-aecf-acde48001122'}


In [21]:
# Define the run parameters.  
# If you are using historian, you will need to search for this time period in Grafana dashboard to view results.
start_dt = datetime.datetime(2017, 9, 26, 0, 0, 0)
end_dt = datetime.datetime(2017, 9, 27, 0, 0, 0)

# For external_clock == true, API calls are used to advance the model.  
# If external_clock == false, Alfalfa will handle advancing the model according to a specified timescale (timescale 1 => realtime)
params = {
    "external_clock": "true",
    "start_datetime": start_dt,
    "end_datetime": end_dt
}

In [22]:
# Start simulations.  since we're using external clock, will take through warmup and stop
for i in model_ids.values():
    print(i)
    ac.start(i, **params)

d5da77f6-c05a-11ec-aecf-acde48001122
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
d9b4e154-c05a-11ec-aecf-acde48001122
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		Current status: Starting
Desired status: Running		C

In [24]:
#run for 1 day  
for i in range(1440):
    # TODO sum across all buildings and persist value
    # TODO check timestep and wriite OpenADR if changed

    # Advance the models.  there is apparently some blocking here at the web layer 
    ac.advance(model_ids.values())

In [ ]:
# Stop the models
ac.stop(m1_id)
ac.stop(m2_id)